# 음악 순위 크롤링 과제 (멜론, 벅스, 지니)

In [1]:
# Jupyter Notebook 설정 : 모든 결과 출력
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

In [2]:
!pip install selenium

In [3]:
!pip install webdriver_manager

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import pandas as pd

## 멜론차트 100위까지 가져와 엑셀파일로 저장하기

In [5]:
melon_df = pd.DataFrame(columns=["rank", "title", "artist"])
url = "https://www.melon.com/chart/index.htm"
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install())) # 웹드라이버 객체 생성
driver.get(url) # 페이지 이동

def build_dataframe(class_name):
    index = len(melon_df)
    tr_tags = driver.find_elements(By.CSS_SELECTOR, "tr." + class_name) # list
    for tr_tag in tr_tags:
        # print(type(tr_tag)) # <class 'selenium.webdriver.remote.webelement.WebElement'>
        rank_tags = tr_tag.find_elements(By.CSS_SELECTOR, "span.rank")
        if (len(rank_tags) > 0):
            title_tags = tr_tag.find_elements(By.CSS_SELECTOR, "div.ellipsis.rank01 a")
            artist_tags = tr_tag.find_elements(By.CSS_SELECTOR, "div.ellipsis.rank02 a")
            artist_str = ""
            if (len(artist_tags) > 0):
                for artist_tag in artist_tags:
                    if (bool(artist_tag.text)):
                        artist_str += ", " + artist_tag.text if bool(artist_str) else artist_tag.text
            melon_df.loc[index] = [rank_tags[0].text, title_tags[0].text, artist_str]
            index += 1

build_dataframe("lst50")
build_dataframe("lst100")
driver.quit()
melon_df.to_excel("site_melon.xlsx", index=False, columns=["rank", "title", "artist"])
melon_df

,rank,title,artist
0,1,Good Goodbye,화사 (HWASA)
1,2,타임캡슐,다비치
2,3,Blue Valentine,NMIXX
3,4,ONE MORE TIME,ALLDAY PROJECT
4,5,Golden,"HUNTR/X, EJAE, AUDREY NUNA, REI AMI, KPop Demo..."
...,...,...,...
95,96,눈이 오잖아(Feat.헤이즈),이무진
96,97,비가 와서,임영웅
97,98,Santa Tell Me,Ariana Grande
98,99,PO￦ER,G-DRAGON


## 벅스차트 100위까지 가져와 엑셀파일로 저장하기

In [6]:
bugs_df = pd.DataFrame(columns=["rank", "title", "artist"])
url = "https://music.bugs.co.kr/chart/track/realtime/total?wl_ref=M_contents_03_01"
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install())) # 웹드라이버 객체 생성
driver.get(url) # 페이지 이동

index = 0
tr_tags = driver.find_elements(By.CSS_SELECTOR, "table.list.trackList.byChart tr")
for tr_tag in tr_tags:
    rank_tags = tr_tag.find_elements(By.CSS_SELECTOR, "div.ranking > strong")
    if (len(rank_tags) > 0):
        title_tags = tr_tag.find_elements(By.CSS_SELECTOR, "p.title > a")
        artist_tags = tr_tag.find_elements(By.CSS_SELECTOR, "p.artist > a")
        artist_str = artist_tags[0].text
        if (len(artist_tags) > 1):
            temp = artist_tags[1].get_attribute("onclick")
            artists_list = temp.split("\\n")
            for i in range(1, len(artists_list)):
                sindex = artists_list[i].find("||")
                artist_str += ", " + artists_list[i][:sindex]
        bugs_df.loc[index] = [rank_tags[0].text, title_tags[0].text, artist_str]
        index += 1

driver.quit()
bugs_df.to_excel("site_bugs.xlsx", index=False, columns=["rank", "title", "artist"])
bugs_df

,rank,title,artist
0,1,Good Goodbye,화사 (HWASA)
1,2,ONE MORE TIME,ALLDAY PROJECT
2,3,SPAGHETTI (feat. j-hope of BTS),"LE SSERAFIM (르세라핌), j-hope"
3,4,Blue Valentine,NMIXX
4,5,Golden,"HUNTR/X, EJAE, AUDREY NUNA, REI AMI, KPop Demo..."
...,...,...,...
95,96,Attention,NewJeans
96,97,Do It,Stray Kids (스트레이 키즈)
97,98,나는 아픈 건 딱 질색이니까,i-dle (아이들)
98,99,BURNING UP,MEOVV (미야오)


## 지니차트 100위까지 가져와 엑셀파일로 저장하기

In [ ]:
import time
import datetime as DT
genie_df = pd.DataFrame(columns=["rank", "title", "artist"])
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install())) # 웹드라이버 객체 생성

index = 0
for i in range(2):
    url = "https://www.genie.co.kr/chart/top200?ditc=D&ymd={0}&hh=12&rtm=Y&pg={1}".format(DT.datetime.now().strftime("%Y%m%d"), i+1)
    driver.get(url) # 페이지 이동
    tr_tags = driver.find_elements(By.CSS_SELECTOR, "tr.list")
    for tr_tag in tr_tags:
        rank_tags = tr_tag.find_elements(By.CSS_SELECTOR, "td.number")
        if (len(rank_tags) > 0):
            rank = rank_tags[0].text.split("\n")[0]
            title_tags = tr_tag.find_elements(By.CSS_SELECTOR, "td.info > a.title")
            title = title_tags[0].text.strip()
            artist_tags = tr_tag.find_elements(By.CSS_SELECTOR, "td.info > a.artist")
            artist = artist_tags[0].text.strip().replace(" & ", ", ")
        genie_df.loc[index] = [rank, title, artist]
        index += 1
    time.sleep(1)

driver.quit()
genie_df.to_excel("site_genie.xlsx", index=False, columns=["rank", "title", "artist"])
genie_df

,rank,title,artist
0,1,Good Goodbye,화사 (HWASA)
1,2,타임캡슐,다비치
2,3,Golden,"HUNTR/X, EJAE, Audrey Nuna, REI AMI, KPop Demo..."
3,4,Drowning,WOODZ
4,5,Blue Valentine,NMIXX
...,...,...,...
95,96,How It's Done,"HUNTR/X, EJAE, Audrey Nuna, REI AMI, KPop Demo..."
96,97,헤어지자 말해요,박재정
97,98,ATTITUDE,IVE (아이브)
98,99,눈물참기,QWER


## 통합 엑셀파일로 저장하기 (멜론, 벅스, 지니)

In [8]:
melon_df['site'] = 'melon'
bugs_df['site'] = 'bugs'
genie_df['site'] = 'genie'
total_df = pd.concat([melon_df, bugs_df, genie_df])
total_df.to_excel("site_total.xlsx", index=False)

# END